**AULA 06 - SQLite**

In [1]:
library(RSQLite)

In [21]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::ident()    masks dbplyr::ident()
✖ dplyr::lag()      masks stats::lag()
✖ .GlobalEnv::sql() masks dplyr::sql(), dbplyr::sql()


dbConnect recebe dois argumentos: 
1) o formato do banco de dados que iremos conectar (driver);
2) o caminho para o arquivo que iremos trabalhar 

In [2]:
fname = file.path("ME315", "disco", "disco.db")
file.exists(fname)

[1] TRUE

In [3]:
conn = dbConnect(SQLite(), fname)
conn

<SQLiteConnection>
  Path: /srv/data/ME315/disco/disco.db
  Extensions: TRUE

*Vendo tabelas e colunas*

In [5]:
# Tabelas existentes no banco de dados
dbListTables(conn)

# Colunas na tabela albums
dbListFields(conn, 'albums')

[1] "albums"          "artists"         "customers"       "employees"      
 [5] "genres"          "invoice_items"   "invoices"        "media_types"    
 [9] "playlist_track"  "playlists"       "sqlite_sequence" "sqlite_stat1"   
[13] "tracks"

[1] "AlbumId"  "Title"    "ArtistId"

*Selecionando registros*

O comando dbGetQuery consulta o banco de dados, extrai os resultados solicitados. A sintaxe é 
**dbGetQuery(<conexao>, <chamada SQL>)**.

In [6]:
# Extraia da tabela albums todas as colunas e todas as linhas e armazene-os em um objeto chamado album_db

album_db = dbGetQuery(conn, 'SELECT * FROM albums')
head(album_db)

AlbumId,Title,ArtistId
<int>,<chr>,<int>
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4


*Comandos para seleção*

O comando SELECT é o comando mais utilizado em SQL, e pode ser combinado com uma série de argumentos:

**ORDER BY:** ordernar o resultado;

**DISTINCT:** pesquisar por linhas únicas;

**WHERE:** filtrar linhas;

**LIMIT:** restringir o número de linhas do resultado;

**INNER JOIN/LEFT JOIN:** consultar múltiplas tabelas;

**GROUP BY:** agrupar e aplicar funções para agregação nos grupos;

**HAVING:** filtrar em grupos;

In [7]:
# Selecione as colunas trackid, name, composer e unitprice. Ordene o objeto resultante por unitprice.

sql = paste('SELECT trackid, name', 'FROM tracks ORDER BY name')
res = dbGetQuery(conn, sql)
head(res)

TrackId,Name
<int>,<chr>
3027,"""40"""
2918,"""?"""
3412,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525: I. Allegro"
109,#1 Zero
3254,#9 Dream
602,'Round Midnight


Como visto no exemplo acima, separamos as colunas que usamos como parâmetro para seleção por vírgulas.

In [8]:
# Limitando-se a 5 registros, após ordenação por nome, quais são os nomes, ID de álbum e ID de mídia de músicas com mídias de tipo 1 ou 2?

sql = paste('SELECT name, albumid, mediatypeid FROM tracks', 'WHERE mediatypeid IN (1, 2)', 'ORDER BY name LIMIT 5')
dbGetQuery(conn, sql)

Name,AlbumId,MediaTypeId
<chr>,<int>,<int>
"""40""",239,1
"""Eine Kleine Nachtmusik"" Serenade In G, K. 525: I. Allegro",281,2
#1 Zero,11,1
#9 Dream,255,2
'Round Midnight,48,1


In [9]:
# Quais são as músicas e identificadores de faixa e álbum produzidas pelo artista que identificador 12?

sql = paste('SELECT trackid, name, albumid FROM tracks', 'WHERE albumid IN', '(SELECT albumid FROM albums WHERE artistid==12)', 'LIMIT 5')
dbGetQuery(conn, sql)

TrackId,Name,AlbumId
<int>,<chr>,<int>
149,Black Sabbath,16
150,The Wizard,16
151,Behind The Wall Of Sleep,16
152,N.I.B.,16
153,Evil Woman,16


In [10]:
#Quais são as faixas cujos nomes começam com qualquer caracter seguido de 'ere' e terminam com qualquer expressão?

sql = "SELECT trackid, name FROM tracks WHERE name GLOB '?ere*'"
dbGetQuery(conn, sql)[1:5,]

TrackId,Name
<int>,<chr>
324,Pererê
1132,Serenity
1452,Were Do We Go From Here
1740,Sereia
2198,Jeremy


*Funções de Agregação*

AVG: *AVG([ALL | DISTINCT] expressao)* calcula a média de todos os valores não-nulos ou dos valores distintos;

COUNT: *COUNT([ALL | DISTINCT] expressao)* realiza a contagem de todos registros;

MAX, MIN, SUM funcionam de maneira análoga às funções anteriores.

**AULA 07 - SQLite: inserindo dados + dbplyr**

In [16]:
library(dbplyr)

*Inserindo uma tabela*

A sintaxe para criar uma tabela vazia, no SQLite, é através do comando CREATE TABLE nome (col1 tipo, col2 tipo2, ...)

In [19]:
dbExecute(conn, "CREATE TABLE instruments (AlbumId INTEGER, TrackId INTEGER, ElectricGuitar INTEGER,
Singer INTEGER, Trumpet INTEGER)")

ERROR: Error: attempt to write a readonly database


*Removendo uma tabela*

In [24]:
dbListFields(conn, 'instruments')

ERROR: Error: no such table: instruments


In [14]:
dbExecute(conn, "DROP TABLE instruments")

ERROR: Error: no such table: instruments


*Incluindo linhas numa tabela*

Voltando ao caso de instrumentos, suponha que eu tenha criado a tabela "instruments", e quero completá-la com alguma informação. Uma maneira de fazê-lo é usando o comando INSERT INTO tabela VALUES (...)

In [15]:
dbExecute(conn,
          "INSERT INTO instruments
VALUES ('85', '1075' , 0, 1, 0), ('85', '1078' , 0, 1, 0); ")

ERROR: Error: no such table: instruments


O comando acima adiciona duas linhas com cada valor em sua respectiva coluna.

*Inserindo uma tabela diretamente*

O parâmetro append concatena uma tabela nova a dados existentes. Por exemplo:

In [27]:
theAvgCar <- mtcars %>%
summarise_all(function(x) round(mean(x), 2))

dbWriteTable(conn, "mtcars", theAvgCar, append = TRUE)
dbGetQuery(conn, "SELECT * FROM mtcars") %>% tail(3)

ERROR: Error: attempt to write a readonly database


**Leitura de SQL em chunks**

Em certo sentido, dbGetQuery() é um atalho para dbSendQuery() seguido de dbFetch() (e dbClearResult()). Uma vantagem de usar a sintaxe mais longa é que podemos ler dados em chunks:

In [28]:
res <- dbSendQuery(conn, "SELECT * FROM mtcars WHERE cyl = 4")
while(!dbHasCompleted(res)){
  chunk <- dbFetch(res, n = 5)
  print(nrow(chunk))
}

dbClearResult(res)

ERROR: Error: no such table: mtcars


O exemplo acima só guarda o último chunk, e não é muito eficiente. Pode ser suficiente se estivermos guardando os resultados com dbWriteTable e append = TRUE.

In [ ]:
# Exemplo:

n <- 1000000
f <- file("chr1.ld")
open(f)
data <- read_table(f, nrow = n, header = TRUE)

con_data <- dbConnect("SQLite", dbname = "R2")
while (nrow(data) == n){
  dbWriteTable(con_data, data, name = "ch1", append = TRUE, header = TRUE)
  data <- read_table(f, nrow = n, header = TRUE)
}
close(f)
if (nrow(data) != 0){
  dbWriteTable(con_data, data, name = "ch1", append = TRUE)
}

*Fechando conexões*

É importante encerrar suas conexões com dbDisconnect(). Além disso, vou remover a cópia que fiz da database disco.db.

In [ ]:
dbDisconnect(db)
if("discoCopy.db" %in% list.files("../dados/")){
file.remove("../dados/discoCopy.db")
}

In [ ]:
*Verbos do dplyr disponíveis*

In [ ]:
meanTracks <- tracks %>%
group_by(AlbumId) %>%
summarise(AvLen = mean(Milliseconds, na.rm = TRUE), AvCost = mean(UnitPrice, na.rm = TRUE))

meanTracks